In [1]:
import nltk
import pickle
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
pTrees = nltk.corpus.treebank.parsed_sents()
%time len(pTrees) #hm... so counting through all of the trees takes about 5 seconds, but this isn't even touching any part of the tree.
print(pTrees[0])

Wall time: 4.63 s


3914

(S
  (NP-SBJ
    (NP (NNP Pierre) (NNP Vinken))
    (, ,)
    (ADJP (NP (CD 61) (NNS years)) (JJ old))
    (, ,))
  (VP
    (MD will)
    (VP
      (VB join)
      (NP (DT the) (NN board))
      (PP-CLR (IN as) (NP (DT a) (JJ nonexecutive) (NN director)))
      (NP-TMP (NNP Nov.) (CD 29))))
  (. .))


In [3]:
pTreeProductions = []
for tree in pTrees:
    pTreeProductions.append(tree.productions())

In [4]:
len(pTreeProductions)
pTreeProductions[0]

3914

[S -> NP-SBJ VP .,
 NP-SBJ -> NP , ADJP ,,
 NP -> NNP NNP,
 NNP -> 'Pierre',
 NNP -> 'Vinken',
 , -> ',',
 ADJP -> NP JJ,
 NP -> CD NNS,
 CD -> '61',
 NNS -> 'years',
 JJ -> 'old',
 , -> ',',
 VP -> MD VP,
 MD -> 'will',
 VP -> VB NP PP-CLR NP-TMP,
 VB -> 'join',
 NP -> DT NN,
 DT -> 'the',
 NN -> 'board',
 PP-CLR -> IN NP,
 IN -> 'as',
 NP -> DT JJ NN,
 DT -> 'a',
 JJ -> 'nonexecutive',
 NN -> 'director',
 NP-TMP -> NNP CD,
 NNP -> 'Nov.',
 CD -> '29',
 . -> '.']

In [5]:
def getAllInstances(wordType):
    instances = []
    for tree in pTreeProductions:
        for rule in tree:
            text = str(rule)
            left, right = text.split(" -> ")
            #print("Left, Rule:")
            #print(left)
            #print(rule)
            #print()
            if(left == wordType):
                instances.append(rule)
    return instances

In [6]:
adjs = getAllInstances('JJ')
len(adjs)

5834

In [19]:
file = open('cats.pkl','rb') 
cats = pickle.load(file)

In [20]:
for cat in cats:
    if(cat == 'NN'):
        print(cat)

NN


In [98]:
cat_lists = [] #list of lists
for cat in cats:
    print("Loading all instances of: ", cat)
    %time cat_lists.append(getAllInstances(cat))

Loading all instances of:  ,
Wall time: 959 ms
Loading all instances of:  ROOT
Wall time: 1.16 s
Loading all instances of:  NP-TMP
Wall time: 1.05 s
Loading all instances of:  QP
Wall time: 1.05 s
Loading all instances of:  PRT
Wall time: 977 ms
Loading all instances of:  WP
Wall time: 1.06 s
Loading all instances of:  PRP$
Wall time: 980 ms
Loading all instances of:  WHNP
Wall time: 956 ms
Loading all instances of:  PP
Wall time: 1.01 s
Loading all instances of:  WRB
Wall time: 1e+03 ms
Loading all instances of:  FRAG
Wall time: 1.03 s
Loading all instances of:  DT
Wall time: 1.34 s
Loading all instances of:  JJS
Wall time: 1.06 s
Loading all instances of:  MD
Wall time: 1.07 s
Loading all instances of:  VBZ
Wall time: 977 ms
Loading all instances of:  CC
Wall time: 1.05 s
Loading all instances of:  EX
Wall time: 985 ms
Loading all instances of:  RBS
Wall time: 991 ms
Loading all instances of:  NN
Wall time: 1 s
Loading all instances of:  S
Wall time: 995 ms
Loading all instances of: 

In [96]:
def getWordProb(word, cat): #this traverses outer list
    count = 0
    for c in cat_lists:
        #print(cat_lists[count][0])
        text = str(cat_lists[count][0])
        if(cat == text.split(" -> ")[0]):
            prob = doMath(word, cat_lists[c])
        count+=1
    return prob

def doMath(word, productions): #given inner list, counts them
    hits = 0
    total = 0
    for rule in productions:
        text = str(rule)
        left, right = text.split(' -> ')
        if(word in right): #does this need revision? doing this to avoid '' nonsense
            hit+=1
        count+=1
    return hits/total

In [97]:
getWordProb('the', 'DT')

TypeError: list indices must be integers or slices, not list